In [8]:
NAVER_URL = "https://comic.naver.com/index.nhn"
TOP_URL = "https://comic.naver.com/webtoon/weekday.nhn"

In [9]:
import requests as rq
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs, urljoin
import json
import pandas as pd

In [10]:
def save(data,file_name):
    file = open(file_name,'a')
    file.write(data+ "\n")

In [11]:
def get_dailywebtoons():
    webtoon_main_url = TOP_URL
    res = rq.get(webtoon_main_url)
    main_soup=BeautifulSoup(res.content,'lxml')
    webtoon_links = [{"title": a_tag.get("title"),"link": urljoin(NAVER_URL,a_tag.get("href"))}
                    for a_tag in main_soup.select(".daily_all a.title")]
    return webtoon_links

In [12]:
def make_link(webtoon_url, page_count):
    return webtoon_url +'&page='+str(page_count)

In [13]:
def get_all_webtoon(webtoon, is_save):
    '''
    해당 웹 툰의  1화 ~ 마지막까지 수집
    '''
    page_count = 1
    is_unlast = True
 
    target_webtoons = list()
    webtoon_url = webtoon['link']
    webtoon_title = webtoon['title']
 
    while is_unlast:
        link = make_link(webtoon_url, page_count)
        target_webtoon_res = rq.get(link)
        webtoon_soup = BeautifulSoup(target_webtoon_res.content, 'lxml')
        a_tags = webtoon_soup.select('.viewList td.title a')
 
        for a_tag in a_tags:
            t = a_tag.text.replace('\n', '').replace('\r', '').replace('\t', '')
            h = urljoin(NAVER_URL, a_tag.get('href'))
 
            if h not in target_webtoons:
                target_webtoons.append(h)
            else:
                is_unlast = False
 
        page_count += 1
 
    if is_save:
        for webtoon in target_webtoons:
            save(webtoon_title + ':' + webtoon, 'all_webtoons.txt')
 
    return target_webtoons
 

In [20]:
def data_parse(soup, url):
    star_total=[]   ;txt = []   ;dictt = {}  
    title_txt = soup.find("div",{"class" : "detail"}).text
    star = soup.select('#topPointTotalNumber')[0].text
    starperson = soup.find('span', {"class" : "pointTotalPerson"}).text  
    title = soup.title.text.split(':')[0]
    print(star,starperson)
    
    titleId = str(parse_qs(urlparse(url).query)['titleId'][0])
    no = str(parse_qs(urlparse(url).query)['no'][0])
    
    star_total.append([titleId,no,star,starperson])

    comment_url = NAVER_URL + '/comment/comment.nhn?titleId=' + titleId + '&no=' + no
    objectId = titleId + '_' + no          
    

    page_count = 1 
    comments_total = 0 
 
    u = 'http://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery1113012327623800394427_1489937311100&lang=ko&country=KR&objectId=' +objectId+ '&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&sort=NEW&_=1489937311112'
  
    while True:
        comment_url = make_link(u, page_count)
            
        header = {
            "Host": "apis.naver.com",
            "Referer": "http://comic.naver.com/comment/comment.nhn?titleId=" + titleId + "&no=" + no,
            "Content-Type": "application/javascript"
        }
        
        res = rq.get(comment_url, headers = header)
        soup = BeautifulSoup(res.content, 'lxml')
        try:
           
            
            content_text = soup.select('p')[0].text
            one = content_text.find('(') + 1
            two = content_text.find(');')
            content = json.loads(content_text[one:two])
 
            comments = content['result']['commentList']
    
            for comment in comments:
                print(comment['contents'],comment['modTime'])
                txt.append([comment['contents'],comment['modTime'].split("T")[0],comment['modTime'].split("T")[1].split("+")[0]])                
            
            if not len(comments):
                break
            else:
                page_count += 1
                dictt.update({str(page_count-1)+"페이지" : [txt,len(comments)]})
                comments_total = comments_total +len(comments)
                txt=[]
                print(dictt)
                
        except:
            pass
        
    txt_total = pd.DataFrame(dictt,star_total)
    txt_total.to_csv('webtoon_txt'+str(objectId)+str(comments_total)+'.csv', index=False, header=False)

 
 
if __name__ == "__main__":
    webtoons = get_dailywebtoons()
    for webtoon in webtoons:
        target_webtoons = get_all_webtoon(webtoon, False)
        for webtoon_page in target_webtoons:
            res = rq.get(webtoon_page)
            webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
            data_parse(webtoon_page_soup, webtoon_page)


{'title': '신의 탑', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=183559&weekday=mon'}
['https://comic.naver.com/webtoon/detail.nhn?titleId=183559&no=427&weekday=mon', 'https://comic.naver.com/webtoon/detail.nhn?titleId=183559&no=426&weekday=mon', 'https://comic.naver.com/webtoon/detail.nhn?titleId=183559&no=425&weekday=mon', 'https://comic.naver.com/webtoon/detail.nhn?titleId=183559&no=424&weekday=mon', 'https://comic.naver.com/webtoon/detail.nhn?titleId=183559&no=423&weekday=mon', 'https://comic.naver.com/webtoon/detail.nhn?titleId=183559&no=422&weekday=mon', 'https://comic.naver.com/webtoon/detail.nhn?titleId=183559&no=421&weekday=mon', 'https://comic.naver.com/webtoon/detail.nhn?titleId=183559&no=420&weekday=mon', 'https://comic.naver.com/webtoon/detail.nhn?titleId=183559&no=419&weekday=mon', 'https://comic.naver.com/webtoon/detail.nhn?titleId=183559&no=418&weekday=mon', 'https://comic.naver.com/webtoon/detail.nhn?titleId=183559&no=417&weekday=mon', 'https://comic.naver.c

KeyboardInterrupt: 

In [31]:
def data_parse(soup, url):
    star_total=[]   ;txt = []   ;dictt = {}  
    title_txt = soup.find("div",{"class" : "detail"}).text
    star = soup.select('#topPointTotalNumber')[0].text
    starperson = soup.find('span', {"class" : "pointTotalPerson"}).text  
    title = soup.title.text.split(':')[0]
    print(star,starperson)
    
    titleId = str(parse_qs(urlparse(url).query)['titleId'][0])
    no = str(parse_qs(urlparse(url).query)['no'][0])
    
    star_total.append([titleId,no,star,starperson])

    comment_url = NAVER_URL + '/comment/comment.nhn?titleId=' + titleId + '&no=' + no
    objectId = titleId + '_' + no          
    

    page_count = 1 
    comments_total = 0 
 
    u = 'http://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery1113012327623800394427_1489937311100&lang=ko&country=KR&objectId=' +objectId+ '&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&sort=NEW&_=1489937311112'
  
    while True:
        comment_url = make_link(u, page_count)
            
        header = {
            "Host": "apis.naver.com",
            "Referer": "http://comic.naver.com/comment/comment.nhn?titleId=" + titleId + "&no=" + no,
            "Content-Type": "application/javascript"
        }
        
        res = rq.get(comment_url, headers = header)
        soup = BeautifulSoup(res.content, 'lxml')
        try:
           
            
            content_text = soup.select('p')[0].text
            one = content_text.find('(') + 1
            two = content_text.find(');')
            content = json.loads(content_text[one:two])
 
            comments = content['result']['commentList']
    
            for comment in comments:
                print(comment['contents'],comment['modTime'])
                txt.append([comment['contents'],comment['modTime'].split("T")[0],comment['modTime'].split("T")[1].split("+")[0]])                
            
            if not len(comments):
                break
            else:
                page_count += 1
                dictt.update({str(page_count-1)+"페이지" : [txt,len(comments)]})
                comments_total = comments_total +len(comments)
                txt=[]
                print(dictt)
                
        except:
            pass
        
    txt_total = pd.DataFrame(dictt,star_total)
    txt_total.to_csv('webtoon_txt'+str(objectId)+str(comments_total)+'.csv', index=False, header=False,)

 
 
if __name__ == "__main__":
    webtoons = get_dailywebtoons()
    target_webtoons = get_all_webtoon(webtoons[0], False)
    #print(target_webtoons)
    for webtoon_page in target_webtoons:
        res = rq.get(webtoon_page)
        webtoon_page_soup = BeautifulSoup(res.content, 'lxml')
        data_parse(webtoon_page_soup, webtoon_page)


9.97 (참여 23798)
레드스컬이 왜 저기 가있냐? 2019-05-01T17:18:07+0900
야마가 댕댕못찾은이유 어딨죠?
왜 난 안보이지... 2019-05-01T17:01:54+0900
1위당 2019-05-01T16:23:12+0900
쟤 빨강이 악령아님?...초반에나왐ㅅ던애 2019-05-01T14:30:57+0900
스크롤 실화인가 ㄷㄷ 2019-05-01T12:19:17+0900
악어 졸귀다 2019-05-01T11:39:46+0900
미리보기 나 안들어가짐 2019-05-01T11:39:28+0900
이쯤되니 엔드로시랑 화련이랑 쿤란이네랑 왕난이네랑 다 어디있고 뭐하고있는지 궁금하다 ㅎㅅㅎ 2019-05-01T11:31:22+0900
댕댕보다 댕댕친구가 초 강력할꺼 같다 2019-05-01T11:12:35+0900
미리보기 개빡치네 2019-05-01T11:09:05+0900
나도 미리보기 오류.... 2019-05-01T11:02:50+0900
미리보기 뭐가 취소됐다는 거지... 2019-05-01T10:46:15+0900
담편 왜 오류뜨냐 2019-05-01T10:14:52+0900
9화 어제 결제하고 잠들었는데 왜 오류나고 안떠;; 2019-05-01T10:09:57+0900
다음편 결제해서 보려는데 오류뜬다... 2019-05-01T09:55:45+0900
{'1페이지': [[['레드스컬이 왜 저기 가있냐?', '2019-05-01', '17:18:07'], ['야마가 댕댕못찾은이유 어딨죠?\n왜 난 안보이지...', '2019-05-01', '17:01:54'], ['1위당', '2019-05-01', '16:23:12'], ['쟤 빨강이 악령아님?...초반에나왐ㅅ던애', '2019-05-01', '14:30:57'], ['스크롤 실화인가 ㄷㄷ', '2019-05-01', '12:19:17'], ['악어 졸귀다', '2019-05-01', '11:39:46'], ['미리보기 나 안들어가짐', '2019-05-01', '11:39:

분량 지려버렸다 진짜 2019-04-30T16:27:23+0900
일위고 일위고 2019-04-30T16:27:02+0900
드디어 신의탑이 제자리로 오셨네 2019-04-30T16:19:01+0900
♡ 2019-04-30T16:18:33+0900
에바; 베댓 보신탕이라는 표현은 좀 그런 것 같은데 2019-04-30T16:05:34+0900
커피부분에서 저만웃기나요 ㅋㅋㅋㅋㅋ 2019-04-30T16:01:56+0900
야마가 얼마나 강할가?  뭐 언젠간 밤이 밤바르듯 발라버릴거니까 밤 퐈이팅! 2019-04-30T15:56:21+0900
도데체 몇컷이냐 145컷 맞나? 2019-04-30T15:56:06+0900
신의탑 독자들 ㅈㄴ 똑해, 미리보기로 다 봐놓고도 독자들 예측 보려고 재주행 함 2019-04-30T15:39:07+0900
쟤 악령 아님? 2019-04-30T15:38:32+0900
지금 처음으로 정주행 하고 있는데 어떻게 가면 갈수록 분량이 늘어나죠?이러다 정주행 못할것 같은데ㅜㅜ 2019-04-30T15:20:27+0900
여윽시 밤레기 2019-04-30T15:18:37+0900
132컷 미친.... 사랑해요 진짜❤ 2019-04-30T14:38:30+0900
밤이 대통령 한다면 누구보단 잘 할 것 같아 2019-04-30T14:37:51+0900
광견이 많이 화가 나있군요 ^^ 2019-04-30T14:37:31+0900
{'1페이지': [[['레드스컬이 왜 저기 가있냐?', '2019-05-01', '17:18:07'], ['야마가 댕댕못찾은이유 어딨죠?\n왜 난 안보이지...', '2019-05-01', '17:01:54'], ['1위당', '2019-05-01', '16:23:12'], ['쟤 빨강이 악령아님?...초반에나왐ㅅ던애', '2019-05-01', '14:30:57'], ['스크롤 실화인가 ㄷㄷ', '2019-05-01', '12:19:17'], ['악어 졸귀다', '2019-05-01', '11:39:46'], ['미

말투가 거칠어져...하앍 2019-04-30T11:45:39+0900
아니 와 저기 누가봐도 애플과 미카엘 같이 생긴 애들에 대한 말은 아무도 없지???? 악령 옆에.... 2019-04-30T11:34:13+0900
정말 감사합니다 작가님 2019-04-30T11:06:37+0900
여기서 밤이 말하는 양아치란, 양치기개를 말합니다. 2019-04-30T10:59:41+0900
순진무구 순수힐링 그 자체던 밤이 많이 단단해졌구나...흡ㅠㅜ 2019-04-30T10:56:38+0900
저긴 보르미르인가요? 레드스컬이있넹 2019-04-30T10:56:37+0900
야마 야마돌겠네~~ 2019-04-30T10:05:10+0900
굳굳 2019-04-30T09:40:29+0900
와ㅠㅠㅠㅠ 밤새고 다시 정주행 했다....ㅠㅠㅠㅠ역시 갓 신의탑 2019-04-30T09:36:25+0900
지금 정주행 하면서 2기 초반인데 그림체 왤케 바뀜...? 2019-04-30T09:35:54+0900
밤 뭘 믿고 저러지 2019-04-30T09:32:08+0900
아..진짜 재밋다 2019-04-30T09:07:47+0900
방어와 공격/ 이것은 혹시 밤팀이 댕댕과 같이 삼아제에 참전하여 무언가 딜을 하지 않을까요?
잘보고 갑니다. 2019-04-30T09:00:31+0900
네이버 신의탑 분량이 너무 많아요~
그냥은 못보겠어요~
쿠키 3개로 올려주세요~ 2019-04-30T08:56:01+0900
투견으로 잡히면 살아야하는→ 잡히면 투견으로 살아야하는 으로  문장고쳐야할 거같네요! 2019-04-30T08:52:36+0900
{'1페이지': [[['레드스컬이 왜 저기 가있냐?', '2019-05-01', '17:18:07'], ['야마가 댕댕못찾은이유 어딨죠?\n왜 난 안보이지...', '2019-05-01', '17:01:54'], ['1위당', '2019-05-01', '16:23:12'], ['쟤 빨강이 악령아님?...초반에나왐ㅅ던애', '2019-05-01', '14

분량보소 ㄷㄷ갓꿀잼 신의탑! 2019-04-30T06:33:30+0900
야마가야마돌았넼ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 2019-04-30T06:22:46+0900
1위!!!!
 2019-04-30T06:06:32+0900
쟤도 라크랑 같은 고대종이네 2019-04-30T05:35:29+0900
진짜 최고...... 2019-04-30T04:35:39+0900
와순위변동계속나네 2019-04-30T04:32:16+0900
손목 안아프세요? 내가 다 아프네 2019-04-30T04:25:53+0900
야마가 아니라 밤이 양아치가 된 느낌인뎈ㅋㅋㅋㅋㅋ
에반켈 뭔짓을 했길래 순하디 순했던 밤이 저렇게 됐지..? 2019-04-30T03:52:06+0900
분량실화가 2019-04-30T03:50:34+0900
진짜 어시...;;;자까님,,;;.사랑합니다ㅠㅠㅜ 2019-04-30T03:41:02+0900
아니 근데 왜 에반켈이 야마를 이길 거라고 생각하는 거임 베댓들은...? 에반켈 강한 건 맞는데 야마도 강할 거라고 생각해야되지않나...?ㅎ 2019-04-30T03:19:33+0900
야마는 이미 야마돌았노 2019-04-30T02:36:06+0900
신의탑은 위대하다! 2019-04-30T02:32:55+0900
다시 1위다! 유지시켜야한다 제군들 모두 힘을 보태주게! 2019-04-30T02:32:42+0900
모바일 1위 디 2019-04-30T02:15:54+0900
{'1페이지': [[['레드스컬이 왜 저기 가있냐?', '2019-05-01', '17:18:07'], ['야마가 댕댕못찾은이유 어딨죠?\n왜 난 안보이지...', '2019-05-01', '17:01:54'], ['1위당', '2019-05-01', '16:23:12'], ['쟤 빨강이 악령아님?...초반에나왐ㅅ던애', '2019-05-01', '14:30:57'], ['스크롤 실화인가 ㄷㄷ', '2019-05-01', '12:19:17'], ['악어 졸귀다', '2019-05-01', '11:39:46'

쿤 잘생겨져땈ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 2019-04-30T02:15:23+0900
노력은 헛되지 않는다는걸 작가님을 보고 느끼고 갑니다... 작가님의 성실함은 정말 본받아야 할 점이라고 봐요 2019-04-30T02:14:15+0900
분량이 미쳐버렷다.. 2019-04-30T01:59:03+0900
설득이 아니라 시비 걸러 왔는뎈ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 2019-04-30T01:56:37+0900
4309 2019-04-30T01:54:15+0900
재밌어.. 2019-04-30T01:41:21+0900
작가님 이거는 분량이 양아치아닙니까? 왤케기냐... 2019-04-30T01:20:17+0900
웹툰만 봐서 모르겠는데 에반켈이 야마보다 쎈거야? 배뎃보니 에반켈이 더 쎈거처럼 말하는데 진짜야? 2019-04-30T01:10:47+0900
카라카 슬레이어치고 꽤 신사다운듯. 가도 만났을때 댕댕얘기하는것도 그렇고. 밤이 솔직히 카라카 엄청 개망신준건데... 저정도로만 화내고 넘어가는것도 대단한것 같음. 또 밤이 2년동안 저런 태도였을건데도 계속 같이 뭔가 진행시켜나가고 전쟁준비하는것을 봐도 대인배 기질이 있음. 사실 카라카는 스승을 구하려는 마음만 아니었음 밤이랑 저러고있을 필요도 없는 사람인데.. 2019-04-30T01:05:29+0900
안녕하세요네이버 웹툰 리뷰 카페 [네리카]입니다.
음악이나 영화처럼 다양한 사람들의 리뷰와 분석을 통해서작품 이해도도 높이고 
네이버 웹툰이 단순 감상을 넘어멋진 문화생활이 됐으면 하는 취지에서 만든 카페입니다.
여러분들이 좋아하는 작품에 대해 평소에 생각했던 것들을 글로 남겨주시면 감사하겠습니다. 
추후에는 단행본 증정 이벤트나관련 상품 증정 이벤트도 진행할 예정입니다. 
한 번 들러주세요.                 cafe.naver.com/nwr 2019-04-30T00:52:12+0900
작화.. 왜이렇냐 어시 적당히 써요ㅠㅠ 못봐주겠음 2019-04-30T00:52:04+0900
아 진짜 최초로 쿠키를 구워야하나... 

...

카라카 금발황안인거 아는데 한번쯤은 그 파충류같이 생겨먹은 가면 좀 벗어주지 2019-04-29T22:42:59+0900
아니야..아무리 봐도 에반켈 때문에 미친 것 같아.. 2019-04-29T22:41:50+0900
내용은 좋은데 그림체가 옛날거가더 이쁜것 같아여 2019-04-29T22:40:53+0900
진짜 분량 너무 좋아.. ㅠㅠ 2019-04-29T22:40:34+0900
댕댕 아버지가 도와줄거 같은 눈치다... 2019-04-29T22:29:17+0900
왜 1등 아님 2019-04-29T22:28:01+0900
분량이 진짜 말이 안된다;;; 작가님 괜찮은거 맞죠? 이거 길이가 장난이 아닌데 2019-04-29T22:26:27+0900
밤 존예... 2019-04-29T22:26:01+0900
밤 사랑한다 2019-04-29T22:24:11+0900
창지기 뿐만아니라 나머지두명도 그 배신자들아님?? 막 삐약이랑같이잇던무리들 라헬패거리엿던 아님말거~ 2019-04-29T22:22:42+0900
댕댕은 가도와 닮지 않았네요. 오히려 하켄과 닮은 것 같은데. 아버지가 자기 자식인 줄도 모르고 죽이는 연출이 나올지... 2019-04-29T22:22:29+0900
이 작가 설정, 내용은 진짜 좋은데 그림은 ㅈㄴ 못 그리네 진짜.  2019-04-29T22:20:08+0900
케이지를 케이크로 봤어 2019-04-29T22:18:58+0900
유ㅏ 오늘 밤이랑 쿤 말싸움하는 것도 좋고 밤이 자기 의지 확고히 내비치는 것도 좋고 ㅠㅠㅠㅠㅠ 걍 존잼입니다!! 2019-04-29T22:18:49+0900
솔직히 칼라반 에반켈 보고나서 야마에대해별 흥미가안생김 세봤자 하진성밑일거아냐 2019-04-29T22:18:39+0900
{'1페이지': [[['레드스컬이 왜 저기 가있냐?', '2019-05-01', '17:18:07'], ['야마가 댕댕못찾은이유 어딨죠?\n왜 난 안보이지...', '2019-05-01', '17:01:54'], ['1위당', '2019-

2위아들 3위아들 2019-04-29T21:35:20+0900
분량 돌앗다 2019-04-29T21:34:44+0900
근데 다들 알고 있으셔야 되는 거가 유한성에게 인스턴트 커피는 물이고 보약이고 최고의 음료수입니다 2019-04-29T21:34:32+0900
아들2 아빠2 2019-04-29T21:34:21+0900
역시 신의탑 1위는 과분하죠.. 2019-04-29T21:33:34+0900
갓 siu ^^ㅎㅎ 2019-04-29T21:33:00+0900
말을 가려쓰던 밤이 말을 막 쓰네? 2019-04-29T21:28:24+0900
저거 저거 쿤 배신한 세끼들이 둘이나 있네 2019-04-29T21:26:44+0900
요번밤은ㅡㅡ아저씨같은얼굴로뵈는데
저만그른가여ㅠㅠ
꽃미남밤을돌려주세요ㅠㅠ 2019-04-29T21:26:35+0900
왜 아무도 밤이 연습중이었던 기술에 대해서 말을 안 하시는지... 2019-04-29T21:24:20+0900
그렇죠. 커피는 만병통치약이...  한성아아아아~~~~ 2019-04-29T21:23:38+0900
작가님 마저 게임으로 인해 이웃동네 귀족웹툰과 돌원숭이 웹툰처럼 되면 안 됩니다 2019-04-29T21:21:13+0900
죄다멍멍이같이도생겼네 2019-04-29T21:20:17+0900
~씨 같은 일본식 표현 자제요 2019-04-29T21:19:37+0900
밑에 있는 괴물의 정체는 야마의 형임  2019-04-29T21:12:52+0900
{'1페이지': [[['레드스컬이 왜 저기 가있냐?', '2019-05-01', '17:18:07'], ['야마가 댕댕못찾은이유 어딨죠?\n왜 난 안보이지...', '2019-05-01', '17:01:54'], ['1위당', '2019-05-01', '16:23:12'], ['쟤 빨강이 악령아님?...초반에나왐ㅅ던애', '2019-05-01', '14:30:57'], ['스크롤 실화인가 ㄷㄷ', '2019-05-01', '12:19:17'], ['악어 졸귀다', '2019-

엔드로시 언제나옴 ㅠㅠ 2019-04-29T20:36:14+0900
신의 탑 :
너는 별을 보자며 내 손을 끌었어 
저녁 노을이 진 탑의 문에 걸터앉아
Everytime I look up in the sky
근데 단 한개도 없는 star
팍!(라헬의 뒷통수 브레이크) 2019-04-29T20:27:04+0900
진짜 분량 미쳤,,다른 웹툰 2편 반은 되는거같다 2019-04-29T20:26:48+0900
...아무리봐도 에반켈이..... 애를 너무 강하게 키웠어.... 2019-04-29T20:26:03+0900
악령 몇년만이냐ㅋㅋㅋㅋㅋ 2019-04-29T20:25:49+0900
공감... 개성을 버리고 퀄리티를 택한듯한 작화. 옛날이랑 비교하면 요즘으로 올수록 그림체가 그냥 무난한 소년만화 같다 2019-04-29T20:25:29+0900
양아치 특) 양아치라고 하면 야마 돌아버림 2019-04-29T20:25:17+0900
만점을 안드릴수가 없군요. 감사합니다. 2019-04-29T20:24:25+0900
그림체가 뭔가 좀 ㅠㅠ
옛날에는 뭔가 자연스럽고 느낌상 좋았는데.요즘은 그냥 잘 그리는 쪽으로 가는거 같네요. 옛날 그림체가 훨씬 더 개성이 잘 들어났는데 2019-04-29T20:23:57+0900
유튜브나 네이버 영화 평점처럼 자기 댓글이 맨위에 보이게 하면 안됨? 2019-04-29T20:23:49+0900
갠적인데 전투신 이펙트 퀄은 예전보다 훨 좋아졌지만1부때나 2부 초반부분의 그 역동적인전투씬이나 박진감 긴장감 뭐 이런것들은 예전보다 못하다고 느낌...
어시가 많아져서 그런가싶기도하고 옛날시우작가전투신느낌이 잘 안남.. 2019-04-29T20:19:59+0900
그래도 밤의 사부인 에반켈보다 쎌리가 있냐 에반켈은 밤 성격도 바꾸는 놈이다 2019-04-29T20:18:54+0900
도대체 애 성격을 어케만들어놓은거야 ㅋㅋ 하진성은 그나마 바른교육한거였나 ㅋㅋ 2019-04-29T20:16:26+0900
진짜 점점 스케일은 커지는데 산으로 간

아니 설득하러왔다며..?
양아치라니ㅋㅋㅋㅋㅋ 2019-04-29T19:44:42+0900
묘하게 켄 존이랑 댕댕이랑 닮고 가도랑 루이랑 닮았다 2019-04-29T19:35:21+0900
괜찮아 우리 밤 하고픈거 다해 2019-04-29T19:32:04+0900
와 야마한테 형이있었다니 ㄷㄷ 미쳤네 2019-04-29T19:31:43+0900
와 이제 밤한테.아무도 못 나대겠는딩 ㅌ ㅋ ㅋ ㅋ 2019-04-29T19:29:27+0900
아니 밤 ㅋㅋㅋㅋㅋㅋㅋㅋ살살구슬려서 협상해야지 그렇게 막 들이대면 안해주려고하는게 심린데 ㅋㅋㅋㅋㅋㅋㅋ 2019-04-29T19:29:18+0900
야마 성격이 뭔가 호탕한거 같은데 그런 면을 봤을 땐 밤 마음에 들어할 거 같음 2019-04-29T19:20:34+0900
스승이 바뀌어서 성격이 바뀌엇다기보다 자하드에서 밤과 관련된 아들 다 죽이려고 해서 냉정해지고 지키려하는거  그런거 아닌감 ㅠㅠ 2019-04-29T19:20:24+0900
밤의 너 양아치니덕분에 야마가 야마돌았다 2019-04-29T19:07:51+0900
야마가 야마돔 엌ㅋㅋㅋㅋㅋ 2019-04-29T19:03:07+0900
4컷 ㅗㅜㅑ 2019-04-29T19:01:12+0900
아무리 하이랭커라도 배울게 따로있는거야 그런건 배우지마 2019-04-29T18:53:59+0900
이번화 내려도 내려도 끝이없어...작가님 짱입니다 2019-04-29T18:48:49+0900
야마는 질 나쁜 양아치이고 밤 너는 예의바른 쿤이 되어가는구나..  그러니까 쿤밤 2019-04-29T18:48:29+0900
와우 노잼탑 2019-04-29T18:47:30+0900
{'1페이지': [[['레드스컬이 왜 저기 가있냐?', '2019-05-01', '17:18:07'], ['야마가 댕댕못찾은이유 어딨죠?\n왜 난 안보이지...', '2019-05-01', '17:01:54'], ['1위당', '2019-05-01', '16:23:12'], ['쟤 빨강이 악령아님?...초

KeyboardInterrupt: 

NameError: name 'txt_total' is not defined